In [1]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

# Standard imports
import numpy as np
import pandas as pd
from collections import Counter, OrderedDict
import re
import string
import inspect
import os
import datetime
from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, precision_recall_curve

from lightgbm import LGBMClassifier

from sklearn.metrics import confusion_matrix

In [2]:
import json
import pickle
import joblib
import pandas as pd
from flask import Flask, jsonify, request
from peewee import (
    SqliteDatabase, PostgresqlDatabase, Model, IntegerField,
    FloatField, TextField, IntegrityError
)
from playhouse.shortcuts import model_to_dict

In [3]:
from transformers import transf as tf

,observation_id,Type,Date,Part of a policing operation,Latitude,Longitude,Gender,Age range,Self-defined ethnicity,Officer-defined ethnicity,Legislation,Object of search,Outcome,Outcome linked to object of search,Removal of more than just outer clothing,station
0,34d76816-cfc2-4bdd-b3a2-bf0c40b12689,Person search,2019-12-01T00:00:00+00:00,True,NaN,NaN,Male,18-24,Asian/Asian British - Any other Asian background,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,NaN,False,devon-and-cornwall
1,83c87ec1-e6d4-4bbb-8e0c-159516706359,Person search,2019-12-01T00:09:00+00:00,True,NaN,NaN,Male,18-24,NaN,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,NaN,False,devon-and-cornwall
2,8c2eae4b-035a-492f-92d6-cd27e6c900a1,Person search,2019-12-01T00:10:00+00:00,True,NaN,NaN,Female,18-24,White - English/Welsh/Scottish/Northern Irish/...,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,NaN,False,devon-and-cornwall
3,e63a036a-a728-4efe-8e08-7198f56d6d0a,Person search,2019-12-01T00:10:00+00:00,False,NaN,NaN,Male,18-24,NaN,Asian,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,NaN,False,devon-and-cornwall
4,40c70aca-1f7f-4efe-8c5b-ade0f2e9937d,Person search,2019-12-01T00:12:00+00:00,True,50.368247,-4.126646,Male,18-24,NaN,White,Misuse of Drugs Act 1971 (section 23),Controlled drugs,A no further action disposal,NaN,False,devon-and-cornwall


In [5]:
list(df.head(1).values)

[array(['34d76816-cfc2-4bdd-b3a2-bf0c40b12689', 'Person search',
        '2019-12-01T00:00:00+00:00', True, nan, nan, 'Male', '18-24',
        'Asian/Asian British - Any other Asian background', 'Asian',
        'Misuse of Drugs Act 1971 (section 23)', 'Controlled drugs',
        'A no further action disposal', nan, False, 'devon-and-cornwall'],
       dtype=object)]

In [6]:
df['Part of a policing operation']  =df['Part of a policing operation'].astype('bool')

In [7]:
# Group feaatures 
categorical_features = ['Type','Gender','Age range','Officer-defined ethnicity','Object of search','station']
numeric_features = ['Time (sin)','Time (cos)']

In [8]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

standard_preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


In [9]:
df.columns

Index(['observation_id', 'Type', 'Date', 'Part of a policing operation',
       'Latitude', 'Longitude', 'Gender', 'Age range',
       'Self-defined ethnicity', 'Officer-defined ethnicity', 'Legislation',
       'Object of search', 'Outcome', 'Outcome linked to object of search',
       'Removal of more than just outer clothing', 'station'],
      dtype='object')

In [10]:
target_pipe =  Pipeline(
    steps=[
            ('Filter Metropolitain', tf.Filter_ColumnValue(col='station', value='metropolitan')),
            ('FixNA_OutcomeLinkedSearch', tf.FixNA_OutcomeLinkedSearch()), 
            ('OutcomePositive', tf.Mask_OutcomePositive()),
            ('SearchSuccess', tf.Mask_SearchSuccess()),
            ])

In [11]:
initial_pipe = Pipeline(
    steps=[
            ('Filter Metropolitain', tf.Filter_ColumnValue(col='station', value='metropolitan')),
            ('FixNA_Coordinates', tf.FixNA_Coordinates()),
            ('DateTransformer', tf.DateTransformer())
            ])

In [12]:
pipeline = Pipeline([
        ('Initial Preprocessing Pipeline',initial_pipe),
        ('preprocessor', standard_preprocessor),
        ('model', LGBMClassifier(n_jobs=-1, random_state=42))
]    
)

In [13]:
columns = ['observation_id', 'Type', 'Date', 'Part of a policing operation',
       'Latitude', 'Longitude', 'Gender', 'Age range',
        'Officer-defined ethnicity', 'Legislation',
       'Object of search', 'station']

In [14]:
outcome_columns = ['Outcome', 'Outcome linked to object of search']
df_target = target_pipe.fit_transform(df)
target = df_target['search_success']

In [15]:
df_target.columns

Index(['observation_id', 'Type', 'Date', 'Part of a policing operation',
       'Latitude', 'Longitude', 'Gender', 'Age range',
       'Self-defined ethnicity', 'Officer-defined ethnicity', 'Legislation',
       'Object of search', 'Outcome', 'Outcome linked to object of search',
       'Removal of more than just outer clothing', 'station',
       'outcome_positive', 'search_success'],
      dtype='object')

In [16]:
X_train = df[columns]
y_train = target

In [17]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('Initial Preprocessing Pipeline',
                 Pipeline(steps=[('Filter Metropolitain',
                                  Filter_ColumnValue(col='station',
                                                     value='metropolitan')),
                                 ('FixNA_Coordinates', FixNA_Coordinates()),
                                 ('DateTransformer', DateTransformer())])),
                ('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  ['Time (sin)', 'Time (cos)']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
  

In [18]:
df[columns].dtypes

observation_id                   object
Type                             object
Date                             object
Part of a policing operation       bool
Latitude                        float64
Longitude                       float64
Gender                           object
Age range                        object
Officer-defined ethnicity        object
Legislation                      object
Object of search                 object
station                          object
dtype: object

In [19]:
##Serializing the columns in the correct order

with open( os.path.join("columns.json"), 'w') as fh:
    json.dump(columns, fh)
  
##Serializing the fitted pipeline
joblib.dump(pipeline,os.path.join( "pipeline.pickle")) 

##Serializinf dtypes of columns
with open(os.path.join('dtypes.pickle'), 'wb') as fh:
    pickle.dump(df[columns].dtypes, fh)